USing ChatGPT Tutorial: https://www.youtube.com/watch?v=S3okwVkxDgA

Install
``` bash
pip install pyChatGPT webdriver_manager
```

In [ ]:
from pyChatGPT import ChatGPT
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import os
from dotenv import load_dotenv
load_dotenv()


# Open Canvas

In [ ]:
DRIVER_PATH = './chromedriver'

options = Options()
options.headless = False
options.add_argument("--window-size=1920,1200")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

wait_time = 2

In [ ]:
# give link of quiz to grade
quiz_link = 'https://canvas.cmu.edu/courses/31156/gradebook/speed_grader?assignment_id=513942' 
driver.get(f'{quiz_link}')
time.sleep(wait_time)

try:
    username = driver.find_element(By.ID, "username")
    username.send_keys(os.environ['CANVAS_USERID']) 

    passwd = driver.find_element(By.ID, "passwordinput")
    passwd.send_keys(os.environ['CANVAS_PASSWORD']) 

    button = driver.find_element(By.CLASS_NAME, "loginbutton")
    button.click()
    print("Logged in to Canvas")
except:
    print("Already logged in to Canvas")

# Use .env value OPENAI_SESSION_TOKEN
try:
    openai_session_token = os.environ['OPENAI_SESSION_TOKEN'] # `__Secure-next-auth.session-token` cookie from https://chat.openai.com/chat
    chap_gpt_api = ChatGPT(openai_session_token)  # auth with session token
    print("Logged in to ChatGPT")
    time.sleep(wait_time)
except:
    Exception("Please set SESSION_AUTH_TOKEN in .env file")


In [ ]:
text = driver.find_element(By.ID, "speedgrader_iframe")
text.get_attribute('src')

driver.get(text.get_attribute('src'))
time.sleep(wait_time)
# driver.back()

In [ ]:
question = driver.find_element(By.CLASS_NAME, "question_text").text
question


In [ ]:
answer = driver.find_element(By.CLASS_NAME, "quiz_response_text").text
answer

In [ ]:
prompt = {  'quiz_14': 'Answer yes or no if the answer is correct.', 
            'quiz_17': 'Answer yes or no if the answer made an effort to answer the question.',
            'quiz_20': 'Answer yes or no if the answer is related to two ML specific secruity risks.',
            }
chat_gpt_response = chap_gpt_api.send_message(f'Question: {question} \n\n Answer: {answer} \n\n {prompt["quiz_20"]}')['message']
# chat_gpt_response = chat_gpt_response['message']
chat_gpt_response


In [ ]:
if "yes" in chat_gpt_response.lower():
    print("Correct")
    grade = 1
else:
    print("Incorrect")
    grade = 0
driver.find_element(By.CLASS_NAME, "question_input").send_keys(grade)

In [ ]:
driver.find_element(By.XPATH, "//textarea[starts-with(@id,'question_comment_')]").send_keys(chat_gpt_response)

In [ ]:
driver.find_element(By.ID, 'speed_grader_comment_textarea').send_keys(chat_gpt_response)

In [ ]:
driver.find_element(By.XPATH, "//button[@type='submit']").click()

In [ ]:
driver.back()
# driver.back()

In [ ]:
driver.find_element(By.XPATH, "//button[@id='next-student-button']").click()

In [ ]:
graded_value = driver.find_element(By.ID, "x_of_x_graded").text
graded_value = graded_value.split('/')
int(graded_value[0]), int(graded_value[1])

In [ ]:
while True:
    graded_value = driver.find_element(By.ID, "x_of_x_graded").text
    print(f"Graded: {graded_value}")
    graded_value = graded_value.split('/')
    if int(graded_value[0]) >= int(graded_value[1]):
        break

    student = driver.find_element(By.CLASS_NAME, "ui-selectmenu-item-header").text

    # if driver.find_element(By.ID, "grading-box-extended").get_attribute('value') == '':
    try:
        text = driver.find_element(By.ID, "speedgrader_iframe")
        text.get_attribute('src')

        driver.get(text.get_attribute('src'))
        time.sleep(wait_time)

        if driver.find_element(By.CLASS_NAME, "question_input").get_attribute('value') == '':

            question = driver.find_element(By.CLASS_NAME, "question_text").text
            answer = driver.find_element(By.CLASS_NAME, "quiz_response_text").text

            # time.sleep(wait_time)
            # chat_gpt_response = chap_gpt_api.send_message(f'Question: {question} \n\n Answer: {answer} \n\n {prompt["quiz_20"]}')['message']
            if len(f'Question: {question} \n\n Answer: {answer}') > 200:
                chat_gpt_response = 'yes'
            else:
                chat_gpt_response = 'No, answer too short'

            if "yes" in chat_gpt_response.lower():
                print("Correct")
                grade = 1
            else:
                print("Incorrect")
                grade = 0
                # driver.find_element(By.XPATH, "//textarea[starts-with(@id,'question_comment_')]").send_keys(chat_gpt_response)
            driver.find_element(By.CLASS_NAME, "question_input").send_keys(grade)

            driver.find_element(By.XPATH, "//button[@type='submit']").click()
            time.sleep(wait_time)
            driver.back()
            time.sleep(wait_time)
            grade = driver.find_element(By.CLASS_NAME, "question_input").get_attribute('value')

        driver.back()
        time.sleep(wait_time)
        if grade == '0':
            driver.find_element(By.ID, 'speed_grader_comment_textarea').send_keys(chat_gpt_response)
            driver.find_element(By.ID, "comment_submit_button").click()
            time.sleep(wait_time)

        print(f'Student: {student}, Graded: {grade}')
        
    except:
        print(f'Student: {student} Does not have a submission')
        if driver.find_element(By.ID, "grading-box-extended").get_attribute('value') == '':
            driver.find_element(By.ID, "grading-box-extended").send_keys(0)
            driver.find_element(By.ID, "comment_submit_button").click()
            

    driver.find_element(By.XPATH, "//button[@id='next-student-button']").click()
    time.sleep(wait_time)

print("Done grading all students")
    